In [ ]:
# import libraries and packages

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
stocks = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')
stocks = stocks.drop(['RowId', 'Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'], axis=1)

In [ ]:
def rank_stocks(stock_returns, date):
    
    ranking = pd.DataFrame({'SecuritiesCode': list(stock_returns.keys()), 'Return': list(stock_returns.values())})
    ranking = ranking.sort_values('Return', ascending=False).reset_index(drop=True)
    ranking['Rank'], ranking['Date'] = ranking.index, date
    ranking = ranking.sort_values('SecuritiesCode', ascending=True).reset_index(drop=True)
    
    return ranking

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
stock_returns = {}

for (new_prices, _, _, _, _, sample_prediction) in iter_test:
    current_date = new_prices['Date'].iloc[-1]
    sec_codes = new_prices.loc[:, 'SecuritiesCode'].unique()
    
    todays_stocks = stocks[stocks['Date'] == current_date]
    
    for code in tqdm(sec_codes):
        stock_returns[code] = todays_stocks.loc[todays_stocks['SecuritiesCode'] == code, 'Target'].item()
        
    print('Sample prediction:\n', sample_prediction)
    
    my_prediction = rank_stocks(stock_returns, current_date)
    sample_prediction['Rank'] = list(my_prediction['Rank'])
    
    print('My prediction:\n', sample_prediction)
    
    env.predict(sample_prediction)